In [65]:
#!pip install rtmlib==0.0.13
#!pip install opencv-python==4.11.0.86
#!pip install librosa==0.11.0
#!pip install Cython==3.0.12
# Microsoft Visual C++ 14.0 or greater is required.
# Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
# The pip version of madmon is outdated and the github repo has many changes recently
#!pip uninstall -y madmom
#!conda install git+https://github.com/CPJKU/madmom.git
#!cd madmom && python setup.py develop --user
#!pip install pyaudio==0.2.14
#!pip install beatnet==1.1.1
#!pip install soundfile==0.13.1
# Install FFmpeg in Windows
# Source: https://phoenixnap.com/kb/ffmpeg-windows
#!pip cache purge
#!pip install moviepy==2.1.1
#!pip install onnxruntime==1.21.1

In [48]:
import os

move_filename = '3e7415f'
moves_filepath = os.path.join('outputs', 'moves.json')

In [50]:
!ffmpeg -i inputs/videos/3e7415f.mp4 -ab 160k -ac 2 -ar 44100 -vn processed/videos/3e7415f.wav

ffmpeg version 2025-04-23-git-25b0a8e295-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev3, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx

In [52]:
from BeatNet.BeatNet import BeatNet
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
audio_filepath = os.path.join('processed', 'videos', f'{move_filename}.wav')
estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)
beats = estimator.process(audio_filepath)
beats

array([[0.3 , 2.  ],
       [0.8 , 1.  ],
       [1.28, 2.  ],
       [1.74, 1.  ],
       [2.22, 2.  ],
       [2.7 , 1.  ],
       [3.18, 2.  ],
       [3.66, 1.  ],
       [4.14, 2.  ],
       [4.62, 1.  ],
       [5.1 , 2.  ],
       [5.58, 1.  ],
       [6.06, 2.  ],
       [6.56, 1.  ],
       [7.02, 2.  ],
       [7.5 , 1.  ],
       [8.  , 2.  ],
       [8.46, 1.  ],
       [8.94, 2.  ],
       [9.42, 1.  ]])

In [26]:
''' 
beat_start = 0.22
beat_end = 6.68
num_beats = 16
beat_delta = (beat_end - beat_start) / num_beats
beats = [[round(beat_start + x * beat_delta, 2), x % 8] for x in range(num_beats)]
beats
'''

[[0.22, 0],
 [0.62, 1],
 [1.03, 2],
 [1.43, 3],
 [1.83, 4],
 [2.24, 5],
 [2.64, 6],
 [3.05, 7],
 [3.45, 0],
 [3.85, 1],
 [4.26, 2],
 [4.66, 3],
 [5.06, 4],
 [5.47, 5],
 [5.87, 6],
 [6.28, 7]]

In [58]:
# 9da1eb8 = 4
# 5db13cd = 1
# 5d80c9a = 1
offset = 3

In [60]:
from moviepy import VideoFileClip, TextClip, CompositeVideoClip

video_file = f"inputs/videos/{move_filename}.mp4"
output_file = f"processed/videos/{move_filename}_counts.mp4"


# Load the video
video_clip = VideoFileClip(video_file)

start_counting = True
text_clips = [video_clip]

for index, x in enumerate(beats):
    start_time = x[0]
    end_time = beats[index + 1][0] if index + 1 < len(beats) else x[0]

    start_counting = start_counting or x[1] == 1

    if start_counting:
        # Create the text clip
        text_clip = TextClip(font='arial.ttf', text=f'{(index - offset) % 8 + 1}', font_size=70, color='white', duration=end_time - start_time)
        
        # Set the duration and position of the text clip
        text_clip = text_clip.with_position(('center', 'bottom'))
        
        # Set the start time of the text clip
        text_clip = text_clip.with_start(start_time)

        text_clips.append(text_clip)
        
# Composite the text clip on the video
video_clip = CompositeVideoClip(text_clips)

# Write the output video file
video_clip.write_videofile(output_file, fps=30, codec='libx264')

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'video', 'language': None, 'default': True, 'size': [720, 1280], 'bitrate': 5561, 'fps': 30.1, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.19.100 libx264'}}], 'input_number': 0}], 'duration': 9.78, 'bitrate': 5693, 'start': 0.0, 'default_audio_input_number': 0, 'default_audio_stream_number': 0, 'audio_fps': 44100, 'audio_bitrate': 128, 'default_video_input_number': 0, 'default_video_stream_number': 1, 'video_codec_na

MoviePy - Done.
MoviePy - Writing video processed/videos/3e7415f_counts.mp4



MoviePy - Done !
MoviePy - video ready processed/videos/3e7415f_counts.mp4


In [62]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# After verifying that the counts are Okay, I can save the counts
data[move_filename] = []
section = 1
for index, x in enumerate(beats):
    count = (index - offset) % 8 + 1
    print(count)
    data[move_filename].append({
        'section': section, 
        'count': count,
        'time': x[0],
        'leader_keypoints': [],
        'follower_keypoints': []
    })
    if count == 8:
        section += 1

with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

6
7
8
1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
1


In [64]:
import cv2

start_counting = False
section = 1
for index, x in enumerate(beats):
    count = (index - offset) % 8 + 1
    print(count)
    vidcap = cv2.VideoCapture(video_file)
    vidcap.set(cv2.CAP_PROP_POS_MSEC, int(x[0] * 1000))      # just cue to position
    success, image = vidcap.read()
    if success:
        cv2.imwrite(os.path.join('processed', 'images', f"{move_filename}_section_{section}_count_{count}.jpg"), image)     # save frame as JPEG file

    # Release resources
    vidcap.release()
    cv2.destroyAllWindows()

    if count == 8:
        section += 1

6
7
8
1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
1


In [70]:
import numpy as np
import cv2
import os
from rtmlib import Body, draw_skeleton
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

device = 'cpu'
backend = 'onnxruntime'  # opencv, onnxruntime, openvino

cap = cv2.VideoCapture(0)

openpose_skeleton = False  # True for openpose-style, False for mmpose-style

body = Body(
    pose='rtmo',
    to_openpose=openpose_skeleton,
    mode='performance',  # balanced, performance, lightweight
    backend=backend,
    device=device
)
# TODO: Nowadays, I do this by hand. We need to automate it.
leader_detected_first = [
    True,
    False,
    False,
    True,
    True,
    True,
    True,
    False,
    True,
    True,
    True,
    True,
    False,
    True,
    True,
    True,
    True,
    True,
    True,
    True
]
#leader_detected_first = [False for index in range(len(beats))]
img_index = 0

for filename in os.listdir(os.path.join('processed', 'images')):
    # check if the image ends with png
    if (filename.startswith(move_filename)):
        frame = cv2.imread(os.path.join('processed', 'images', filename), cv2.IMREAD_COLOR)
        keypoints, scores = body(frame)

        if leader_detected_first[img_index]:
            data[move_filename][img_index]["leader_keypoints"] = keypoints[0].tolist()
            if keypoints.shape[0] > 1:
                data[move_filename][img_index]["follower_keypoints"] = keypoints[1].tolist()
        elif np.sum(keypoints) > 0:
            data[move_filename][img_index]["follower_keypoints"] = keypoints[0].tolist()
            if keypoints.shape[0] > 1:
                data[move_filename][img_index]["leader_keypoints"] = keypoints[1].tolist()
        print(filename)
        print(keypoints.shape)
        print(np.sum(keypoints))
        print(keypoints[0])

        img_show = frame.copy()
        img_show = draw_skeleton(
            img_show,
            keypoints,
            #np.array([keypoints[0]]),
            scores,
            openpose_skeleton=openpose_skeleton,
            kpt_thr=0.3,
            line_width=2
        )

        cv2.imwrite(os.path.join('processed', 'keypoints', filename), img_show)
        img_index += 1

# Override the data with the keypoints
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

load C:\Users\user\.cache\rtmlib\hub\checkpoints\rtmo-l_16xb16-600e_body7-640x640-b37118ce_20231211.onnx with onnxruntime backend
3e7415f_section_1_count_6.jpg
(1, 17, 2)
16005.685
[[ 403.25705  251.35431]
 [ 413.30396  233.51291]
 [ 378.3581   233.26321]
 [ 424.76303  254.60162]
 [ 330.95105  257.07263]
 [ 464.34314  336.46417]
 [ 291.27475  384.9604 ]
 [ 565.2539   293.5451 ]
 [ 347.75177  510.04938]
 [ 583.5502   241.36133]
 [ 463.28168  494.7215 ]
 [ 484.1326   631.3359 ]
 [ 371.69733  644.7627 ]
 [ 616.0088   810.09955]
 [ 395.44586  812.0918 ]
 [ 694.81714  993.4211 ]
 [ 380.09003 1014.7868 ]]
3e7415f_section_1_count_7.jpg
(1, 17, 2)
15408.167
[[309.65015 403.46902]
 [319.5631  386.23633]
 [303.13138 392.8686 ]
 [355.5328  380.92172]
 [308.517   392.26144]
 [406.41022 436.70764]
 [303.7552  446.1841 ]
 [495.8241  480.9305 ]
 [257.32632 520.70337]
 [496.3897  429.24103]
 [208.12305 587.9494 ]
 [368.81317 630.90967]
 [345.02942 634.1271 ]
 [250.87762 741.33575]
 [407.82822 790.2122

In [72]:
import numpy as np

def normalize_keypoints_by_bbox_height(keypoints):
    """
    Normalize keypoints so that:
    - The top-left of the bounding box is (0, 0)
    - The bounding box height is 1
    """
    kp = np.array(keypoints)
    
    min_x = np.min(kp[:, 0])
    min_y = np.min(kp[:, 1])
    max_y = np.max(kp[:, 1])
    
    bbox_height = max_y - min_y
    if bbox_height < 1e-5:
        raise ValueError("Bounding box height too small for normalization.")
    
    # Translate to origin and scale
    kp_normalized = (kp - [min_x, min_y]) / bbox_height
    return kp_normalized, bbox_height


def compute_normalized_embeddings(leader_kp, follower_kp):
    leader_kp = np.array(leader_kp)
    follower_kp = np.array(follower_kp)

    # Normalize each person's keypoints by their own bbox height
    norm_leader = np.array([])
    norm_follower = np.array([])
    if np.sum(leader_kp) > 0:
        norm_leader, leader_height = normalize_keypoints_by_bbox_height(leader_kp)
    if np.sum(follower_kp) > 0:
        norm_follower, _ = normalize_keypoints_by_bbox_height(follower_kp)

    rel_vector = np.array([])
    if np.sum(leader_kp) > 0 and np.sum(follower_kp) > 0:
        # Get original centers (before normalization)
        center_leader = np.mean(leader_kp, axis=0)
        center_follower = np.mean(follower_kp, axis=0)
    
        # Compute relative position vector in leader's height units
        rel_vector = (center_follower - center_leader) / leader_height

    return norm_leader.flatten(), norm_follower.flatten(), rel_vector

In [74]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

for index in range(len(data[move_filename])):
    kp_leader = data[move_filename][index]['leader_keypoints']
    kp_follower = data[move_filename][index]['follower_keypoints']
    leader_embedding, follower_embedding, distance_embedding = compute_normalized_embeddings(kp_leader, kp_follower)
    data[move_filename][index]['leader_embedding'] = leader_embedding.tolist()
    data[move_filename][index]['follower_embedding'] = follower_embedding.tolist()
    data[move_filename][index]['distance_embedding'] = distance_embedding.tolist()

# Override the data with the new embeddings
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

In [46]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# Add musicality (TODO: This is manual)
musicality = [
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '#2',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '-'
]
for index in range(len(data[move_filename])):
    data[move_filename][index]['musicality'] = musicality[index]

# Override the data with musicality
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)